## selenium을 이용하여 주유가격 수집

- 매일 1회 수집
- 지역별 검색 사이트에 진입 : opinet.co.kr
- 시/도 -> 시/군/구 -> 읍/면/동 -> wait -> 읍/면/동 변경됨
- '조회' 버튼 클릭
- '엑셀저장' 버튼 클릭
- 이렇게 돌면서 데이터 저장(파일을 모아두고) -> *.xls를 한 번에 목록을 읽어들이고
- pandas에 엑셀을 DataFrame으로 로드하는 함수를 활용하여 변환 후 df의 merge를 이용하여 덩어리로 만들어서 DB에 삽입 or 매번 삽입

In [26]:
from selenium import webdriver as wd
import urllib

In [27]:
target_url = 'http://www.opinet.co.kr/searRgSelect.do'

In [28]:
driver = wd.Chrome('../tool/chromedriver.exe')

In [29]:
# 사이트 접속 & 대기
driver.get(target_url)
import time
time.sleep(5)

In [30]:
driver.get(target_url)

In [31]:
# 시도 정보 추출 및 특정 시도 선택
# select tag이고, 이 요소의 값을 설정하면 시도 값이 변경된다
sido = driver.find_element_by_id('SIDO_NM0')

In [32]:
# 시도 데이터 추출
# select > option들의 속성 중에 value 값 : #SIDO_NM0 > option
for option in driver.find_elements_by_css_selector('#SIDO_NM0 > option') :
    if option.get_attribute('value') :
        print(option.get_attribute('value'))

서울특별시
부산광역시
대구광역시
인천광역시
광주광역시
대전광역시
울산광역시
세종특별자치시
경기도
강원도
충청북도
충청남도
전라북도
전라남도
경상북도
경상남도
제주특별자치도


In [33]:
# 시도 데이터 리스트
sidos = [option.get_attribute('value')
         for option in driver.find_elements_by_css_selector('#SIDO_NM0 > option')
         if option.get_attribute('value')]
sidos

['서울특별시',
 '부산광역시',
 '대구광역시',
 '인천광역시',
 '광주광역시',
 '대전광역시',
 '울산광역시',
 '세종특별자치시',
 '경기도',
 '강원도',
 '충청북도',
 '충청남도',
 '전라북도',
 '전라남도',
 '경상북도',
 '경상남도',
 '제주특별자치도']

In [34]:
# 시도 설정 : send_keys()
# sido.send_keys('서울특별시')

In [35]:
for sido_name in sidos :
    sido.send_keys(sido_name)
    time.sleep(2)

In [36]:
# 시군구 선택
# 사이트 특성상 send_keys()를 수행하면 화면이 전부 reload된다(껌벅인다)
# -> 이전에 잡아두었던 sigungu이 전부 무효화된다
# -> 다시 sigungu 변수를 찾아서 처리해야 한다
sigungu = driver.find_element_by_id('SIGUNGU_NM0')
sigungu.send_keys('제주시')

In [37]:
# 읍면동 선택
dong = driver.find_element_by_id('DONG_NM')
dong.send_keys('아라동')

In [38]:
# '조회' 버튼 클릭
driver.find_element_by_id('searRgSelect').click()

In [39]:
# '엑셀저장' 버튼 클릭
driver.find_element_by_id('glopopd_excel').click()

In [40]:
# driver.close()
# driver.quit()